Topic blacklist generation using supervised learning 

In [27]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(open(r"C:\Users\hp\Downloads\Phishing-Detection-master\new_enron.txt").read())
verb_list = []
noun_list = []
for token in doc:
    if (token.dep_ == 'nsubjpass') or (token.dep_ =='dobj'):
        x=token.head.text
        y=token.text
        verb_list.append(x)
        noun_list.append(y)
        #print(token.text, token.tag_, token.head.text, token.dep_)

In [10]:
dataset = pd.read_csv('datasetfinal3.csv')
dataset.head()



,Unnamed: 0,verb,noun,label,Unnamed: 4
0,0,Suspended,Account,1,NaN
1,1,introducing,method,1,NaN
2,2,subjected,accounts,1,NaN
3,3,request,copies,1,NaN
4,4,expired,account,1,NaN


In [11]:
formatted_dataset = pd.DataFrame()
formatted_dataset['combined'] = dataset['verb'] +" "+dataset['noun']
formatted_dataset['label'] = dataset['label']
formatted_dataset.head()

,combined,label
0,Suspended Account,1
1,introducing method,1
2,subjected accounts,1
3,request copies,1
4,expired account,1


In [12]:
X_train,X_test, y_train,y_test = train_test_split(formatted_dataset['combined'],formatted_dataset['label'],
                                                    train_size = 0.7,
                                                  shuffle=True
                                                  
                                                  )

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [13]:
print(X_train.head())
print(y_train.head())

2146                   set rates
80      authenticate information
389            confirmed account
2325        represent difference
289                perform steps
Name: combined, dtype: object
2146    0
80      1
389     1
2325    0
289     1
Name: label, dtype: int64


In [14]:
vectorizer =CountVectorizer()
X_train_transformed = vectorizer.fit_transform(X_train)

In [15]:
pd.DataFrame(X_train_transformed.toarray(), columns=vectorizer.get_feature_names()).head()

,0ainformation,0ainstructions,0aoneself,ability,absent,accept,acceptance,access,accessed,accomplished,...,work,workout,wrap,writing,www,wwwebaycom,years,you,your,yourself
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
model = MultinomialNB()
%time model.fit(X_train_transformed, y_train)


Wall time: 4 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
X_test_transformed = vectorizer.transform(X_test)
y_pred_class = model.predict(X_test_transformed)
#print(y_pred_class)
prediction = model.predict_proba(X_test_transformed)
#print(prediction,X_test)


In [18]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
results = confusion_matrix(y_test,y_pred_class) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(y_test, y_pred_class))
print('Report : ')
print(classification_report(y_test, y_pred_class))


Confusion Matrix :
[[222  86]
 [ 22 370]]
Accuracy Score : 0.8457142857142858
Report : 
             precision    recall  f1-score   support

          0       0.91      0.72      0.80       308
          1       0.81      0.94      0.87       392

avg / total       0.85      0.85      0.84       700



In [19]:
import pickle
filename = 'blacklist.sav'
pickle.dump(model, open(filename, 'wb'))

In [0]:
X_test_transformed = vectorizer.transform(formatted_dataset['combined'])
y_pred_class = model.predict(X_test_transformed)
#print(y_pred_class)
prediction = model.predict_proba(X_test_transformed)
#print(prediction,X_test)
X_list = list(formatted_dataset['combined'])

In [0]:
#X_list = list(X_test)
len(prediction)
#len(X_list)

2331

In [0]:
#type(prediction[0][1])
count=0
list_pair1 = []
for i in range (0,len(X_list)):
    if(prediction[i][1] >= 0.5 and prediction[i][1]<0.9):
      count+=1
      print(X_list[i] + "   " + str(prediction[i][1]))
      #list_pair1.append(X_list[i])
      
print(count)
      

changed number   0.8088053142649299
notifying us   0.8611958840296433
receive message   0.8757833643093849
reserve right   0.8337117608443464
have reasons   0.520311186359588
receive notifications   0.8942978084887111
review Tips   0.8623929208996892
give password   0.8757833643093849
enable us   0.8611958840296433
expand services   0.8680812123306734
directed site   0.8494057821603556
offered products   0.5562288251426029
valued customer   0.8757833643093849
take actions   0.5655237563185759
prohibited you   0.8705509332709211
includes enrolling   0.7899265934171597
relieve you   0.8705509332709211
pay fees   0.5300911398204255
Submiting inability   0.8826357419597934
enable us   0.8611958840296433
expand services   0.8680812123306734
directed site   0.8494057821603556
make bids   0.7580762609017172
required we   0.7382322707611029
appreciate support   0.7580762609017172
keep place   0.8916051052433708
requires Number   0.6215039880278673
receive notifications   0.8942978084887111
rev

In [0]:
list_pair1 = set(list_pair1)
list_pair2 = list(list_pair1)
len(list_pair2)

260

In [0]:
import pandas as pd
df = pd.DataFrame(list_pair2,columns=['blacklist'])
df.to_csv('blacklist1.csv')

In [0]:
print(prediction)

[[0.64626818 0.35373182]
 [0.64626818 0.35373182]
 [0.97455174 0.02544826]
 ...
 [0.76140969 0.23859031]
 [0.47968881 0.52031119]
 [0.00587507 0.99412493]]


In [0]:
metrics.confusion_matrix(y_test, y_pred_class)


array([[222,  77],
       [ 11, 390]])

# Testing 

In [3]:
import pandas as pd
df = pd.read_csv("blacklist1.csv")

In [12]:
xLower = df["blacklist"].map(lambda x: x.lower())

In [19]:
list1=xLower.tolist()

In [20]:
list1

['receive package',
 'using theory',
 'open browser',
 'detected account',
 'confirm account',
 'directed you',
 'follow the',
 'terminate privileges',
 'maintaining environment',
 'update information',
 'featuring networking',
 'review account',
 'apologize bank',
 'protect security',
 'verify click',
 'provided information',
 'used account',
 'complete form',
 'resolved issue',
 'access accounts',
 'reviewed account',
 'limited access',
 'verify option',
 'selling help',
 'phishing activity',
 'update account',
 'complete construction',
 'follow instructions',
 'trust you',
 'invite you',
 'protect password',
 'complete verification',
 'share password',
 'committed paypal',
 'protecting yourself',
 'verify activity',
 'meets criteria',
 'using bank',
 'visit policy',
 'update notice',
 'suspend account',
 'verify account',
 'noticed activity',
 'selling activitydiscussion',
 'changed information',
 'process request',
 'review preferences',
 'deleted click',
 'apreciate assistance',
 

In [24]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(open(r"C:\Users\hp\Downloads\0_false.txt").read())
_list = []

for token in doc:
    if (token.dep_ == 'nsubjpass') or (token.dep_ =='dobj'):
        x=token.head.text + " " +token.text
        _list.append(x)
        
        #print(token.text, token.tag_, token.head.text, token.dep_)

In [28]:
[x.lower() for x in _list]

['valued member',
 'confirm information',
 'completed this',
 'forced we',
 'suspend account',
 'used it',
 'thank you',
 'appreciate support',
 'keep ebay',
 'keep place',
 'thank you',
 'constitutes acceptance']

In [29]:
check =  any(item in list1 for item in _list)

In [31]:
def intersection(lst1, lst2): 
    lst3 = [value for value in list1 if value in _list] 
    return lst3 
  
print(intersection(list1,_list)) 


['suspend account', 'confirm information', 'constitutes acceptance', 'valued member']


In [30]:
check

True